In [ ]:
%pip install minio

In [1]:
MINIO_HOST = 'localhost'
MINIO_ACCESS_KEY = 'admin'
MINIO_SECRET_KEY = 'admin1234'

In [3]:
from minio import Minio

minio_client = Minio(
    f"{MINIO_HOST}:9000",
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)

In [5]:
minio_client.bucket_exists('user-pics')

False

In [7]:
if not minio_client.bucket_exists('user-pics'):
    minio_client.make_bucket('user-pics')

In [9]:
minio_client.bucket_exists('user-pics')

True

In [11]:
minio_client.list_buckets()

[Bucket('user-pics')]

---

In [13]:
import io

In [15]:
with open('cat.jpg', 'rb') as f:
    data = io.BytesIO(f.read())

result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024
)
print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


In [17]:
data.seek(0)

0

In [19]:
result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat_meta.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024,
    metadata={'creationPlace': 'Moscow, Russia'}
)
print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat_meta.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


In [21]:
from datetime import datetime, timedelta
from minio.retention import Retention
from minio.commonconfig import GOVERNANCE, Tags

In [23]:
data.seek(0)

0

In [25]:
# date = datetime.utcnow().replace(
#     hour=0, minute=0, second=0, microsecond=0,
# ) + timedelta(days=30)

tags = Tags(for_object=True)
tags['usergroup'] = 'teacher'

result = minio_client.put_object(
    bucket_name='user-pics',
    object_name='cat_tags.jpg',
    data=data,
    length=-1,
    part_size=5*1024*1024,
    metadata={'creationPlace': 'Moscow, Russia'},
    tags=tags,
)

print(
    f'created {result.object_name} object; etag: {result.etag}, version-id: {result.version_id}'
)

created cat_tags.jpg object; etag: 9563277ca4f0c8833911b2b70eb3fcec, version-id: None


---

In [28]:
import json

In [ ]:
http://localhost:9000/user-pics/cat.jpg

In [ ]:
minio_client.get_bucket_policy('user-pics')

In [27]:
# https://awspolicygen.s3.amazonaws.com/policygen.html

policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Action": [
        "s3:GetObject"
      ],
      "Effect": "Allow",
      "Resource": "arn:aws:s3:::user-pics/*",
      "Principal": {"AWS": "*"}
    }
  ]
}

minio_client.set_bucket_policy('user-pics', json.dumps(policy))

In [29]:
minio_client.get_bucket_policy('user-pics')

'{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"AWS":["*"]},"Action":["s3:GetObject"],"Resource":["arn:aws:s3:::user-pics/*"]}]}'

---

In [40]:
from minio.commonconfig import SnowballObject

In [ ]:
data.seek(0)
with open('dog.jpeg', 'rb') as f:
    data_dog = io.BytesIO(f.read())

minio_client.upload_snowball_objects(
    'user-pics',
    [
        SnowballObject('many_dog.jpeg', data=data_dog, length=-1),
        SnowballObject('many_cat.jpeg', data=data, length=-1)
    ],
)

---

In [31]:
for i in minio_client.list_objects('user-pics'):
    print(i.object_name)

cat.jpg
cat_meta.jpg
cat_tags.jpg


In [33]:
for i in minio_client.list_objects('user-pics'):
    minio_client.remove_object('user-pics', i.object_name)

In [35]:
for i in minio_client.list_objects('user-pics'):
    print(i.object_name)

---

In [38]:
minio_client.remove_bucket('user-pics')

https://min.io/docs/minio/linux/developers/python/API.html